# Interdisciplinary Project
by Marina Sommer | 11778902

Workflow is as follows:

In [1]:
# import packages
from urllib.request import urlretrieve
import os
import re 
import random
import gzip
import json
from collections import Counter
import time
from warcio import ArchiveIterator
import requests
import pandas as pd
import numpy as np

In [2]:
# 1) select crawl: CC-MAIN-2023-40 and download cluster.idx
crawl_name = "CC-MAIN-2023-40"
top_lvl_domain = "at"   # oder "de"
path1 = 'https://data.commoncrawl.org/cc-index/collections/'
path2 = '/indexes/'
path_ccrawl = path1 + crawl_name + path2
url = path_ccrawl + 'cluster.idx'
print(url)

https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2023-40/indexes/cluster.idx


In [3]:
crawl_dir = os.path.join("S:", "msommer", crawl_name, top_lvl_domain)
if not os.path.exists(crawl_dir):
   os.makedirs(crawl_dir)
cluster_file = os.path.join(crawl_dir, "cluster.txt")
#urlretrieve(url, cluster_file)

In [4]:
# 2) filter cdx files with tld '^at,' or '^de,'
regex = '^' + top_lvl_domain + ','
with open(cluster_file, "rt") as file:     
    cdx_files = []
    for line in file:
        tld = line.split("\t")
        match = re.search(regex, tld[0])  # ^at,
        if match: 
            if not tld[1] in cdx_files:
                cdx_files.append(tld[1])
print(cdx_files)      

['cdx-00002.gz', 'cdx-00003.gz', 'cdx-00004.gz']


In [18]:
# 3) randomly choose cdx files, e.g. cdx-00003.gz -> way too much for de
if len(cdx_files) > 1:
    idx = random.sample(range(0, len(cdx_files)), 1)
    print(idx)
    cdx_files = [cdx_files[i] for i in idx]
    print(cdx_files)

In [8]:
# 4) download cdx files (one takes ~ 2 min)
# wget “https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2022-05/indexes/$i.gz"
for file in cdx_files:
    print(file)
    url = path_ccrawl + file
    filename = os.path.join(crawl_dir, file)
    #urlretrieve(url, filename)

cdx-00003.gz


In [9]:
from urllib.request import urlretrieve
def tryDownload(url, filename, retries=0):
    if retries > 10: 
        print("Download failed.")
        return
    try:
        urlretrieve(url, filename)  
    except:
        time.sleep(1)
        tryDownload(url, filename, retries+1)

In [19]:
# 5) gunzip and open it - one line looks like the following:
warc_files = []
for file in cdx_files:
    filename = os.path.join(crawl_dir, file)
    print(filename)
    with gzip.open(filename, 'rt') as f:
        for line in f:          
            match = re.search(regex, line)  # ^at,
            if match: 
                l = line.split("{\"url\":")
                string = "{\"url\":" + l[1]
                d = json.loads(string)
                warc = d["filename"]
                if int(d["length"]) > 10000: 
                    warc_files.append(warc)
        count_dict = Counter(warc_files)    # 1 - 73 occurrences per warc file
        
        # only download warc files with a lot of occurrences --> maybe download not necessary? --> see below
        wet_files = []
        for key, value in count_dict.items():
            if value >= 50:
                # 6) get filename, convert to wet file and download (WARC: ~ 2 min, WET: 15 sec )
                key = key.replace("/warc/", "/wet/").replace("warc.gz", "warc.wet.gz")
                key_path = key.split("/")[-1]
                filename = os.path.join(crawl_dir, key_path)
                wet_files.append(filename)
                print(filename)
                url = "https://data.commoncrawl.org/" + key
                print(url)
                #tryDownload(url, filename)  
                break
                
    
            
    break

S:msommer\CC-MAIN-2023-40\at\cdx-00003.gz


KeyboardInterrupt: 

warcinfo
request
response
metadata

In [16]:
wet_file = "S:msommer\CC-MAIN-2023-40\\at\CC-MAIN-20231001041719-20231001071719-00247.warc.wet.gz"
fname = wet_file.replace(".warc.wet.gz", "-text.txt")
with open(wet_file, 'rb') as stream, open(fname, 'wb') as f:
    x = 0
    for record in ArchiveIterator(stream):
        if record.rec_type == 'conversion':
            regex = '\.' + top_lvl_domain + '/'
            match = re.search(regex, record.rec_headers.get_header('WARC-Target-URI'))
            length = int(record.rec_headers.get_header('Content-Length'))
            rec_type = record.rec_headers.get_header('Content-Type')
            if match and length > 10000 and rec_type == "text/plain":
                print(record.rec_headers.get_header('WARC-Target-URI'))
                content = record.content_stream().read()
                f.write(content)

                x += 1
        if x > 10:
            break


http://www.oe1iah.at/Hardware/Antennen/Ofenrohr.shtml
https://blog.lehofer.at/2016/07/Tele2.html?m=0
https://industriemagazin.at/fertigen/maersk-nutzt-sap-business-technology-platform-zur-beschleunigung-des-strategischen-wandels/
https://nksp.at/?shpxid=f8814aac-c894-4d69-aad5-031a08f37f93
https://tv.skrapid.at/detail/list/295?videoId=18571
https://www.akad-fernstudium.at/abschluss/weiterbildung/international-business-managerin-china-akad/
https://www.beautygossip.at/2014/03/page/2/
https://www.gear4music.at/de/Perkussion/Zubehoer/Glockenspiel-Zubehoer
https://www.grazwiki.at/Rudolf-Hans-Bartsch-Stra%C3%9Fe_3
https://www.haefele.at/de/produkt/tuerdruecker-garnitur-haefele-startec-modell-ldh-2171-edelstahl/P-00895232/
https://www.ingastro.at/produkt/spind-800x450x1700-mm-2-turig/


In [ ]:
### QUESTION: Wie löse ich das Problem, dass oft nur Überschriften im Text vorkommen?! 
# -> vlt beim einlesen von text file line per line die mit wenigen zeichen vernachlässigen
### QUESTION: Wie stelle ich sicher, dass nur deutsche Webinhalte verwendet werden??
# -> line per line language detection ?? -> sehr zeitaufwendig

In [73]:
from langdetect import detect
fname = "S:msommer\CC-MAIN-2023-40\\at\CC-MAIN-20231001041719-20231001071719-00247-text.txt"
final_fname = fname.replace("text.txt", "finaltext.txt")
with open(fname, "rt", encoding="utf-8") as file, open(final_fname, 'wt', encoding="utf-8") as f:
    for line in file:
        if line.count('.') < 2:
            continue
        try:
            lang = detect(line)
        except:
            lang = "none"
        if lang != "de":
            continue
        f.write(line) 

https://stackoverflow.com/questions/34612691/how-can-one-extract-every-payload-from-warc-wet-gz

The code extracts the first 100 text "payloads" from the .warc.wet.gz file. The 100 created text files have shortened names which correspond to their associated url in the warc.wet.gz file. Many of the urls are too long to be used as as convenient names. The file wet_list_out.txt is generated and provides the correspondence between the shortened .txt file names and the associated urls.
The code works. Of course you need to replace warc_wet_gz with the full path name of your own .warc.wet.gz file. If you want all the payloads, not just the first 100, set num to a very large number. Variable num was tacilty assumed to be 100 in the initial post. Why was it rated as not useful? The problem is solved. If you find issues with it please make specific suggestions. 

In [ ]:
import requests
from warcio.archiveiterator import ArchiveIterator
from requests_file import FileAdapter
def print_f_records(fpath,f_out,num):
    record_count = 0
    conversion_count = 0
    header_count = 0
    splitter = "://"
    mx = num
    s = requests.session()
    s.mount('file://',FileAdapter())
    resp = s.get(fpath, stream=True)
    for record in ArchiveIterator(resp.raw, arc2warc=True):
        record_count +=1
        if record.rec_type == 'warcinfo':
            print(record.raw_stream.read())
        elif record.rec_type == 'conversion':
            conversion_count +=1
            if record.raw_stream.read is not None:
                if record.rec_headers.get_header('Content-Type') == 'text/plain':
                    header_count +=1
                    if header_count > mx:
                        continue
                    print("\n text/plain header no: ", header_count)
                    prefix = f"a{header_count}_"        # filenames will begin with a<header_count>
                    fname1 = (record.rec_headers.get_header('WARC-Target-URI'))
                    fname = fname1.split(splitter)[1]   # get the name part which follows ://
                    fname = prefix + fname[:22] + '.txt'    # keep entire file name to 32 characters or less
                    fname = fname.replace("/", "_")
                    content = record.content_stream().read()
                    f = open(fname, 'wb')               # note the wb; write in binay mode because the content is characters
                    f.write(content)
                    f.close()
                    line = f'{fname1},{fname}' + "\n" # keep a record of the url translations
                    f_out.write(line)
                    print("created text file: ", fname)
                    if header_count == mx:
                        print(f"\n... completed {mx} txt files ... \n")
    print("Number of records: ", record_count)
    print("Number of conversion types: ", conversion_count)
    print("Number of text/plain http_headers: ", header_count)
    print("\nFINISHED")
warc_wet_gz ='file:////home/psl/CCrawl/text_extraction/text_from_wet/CC-MAIN-20230928063033-20230928093033-00593.warc.wet.gz'
print(f'\n warc.wet.gz file is {warc_wet_gz}')
num=100
f_out=open("wet_list_out.txt", 'w')
print_f_records(warc_wet_gz, f_out, num)
f_out.close()